In [1]:
# US Open Pool Scraper

In [18]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import csv

In [19]:
url = "https://cuescore.com/tournament/US+Open+9-ball+Championship+2019/5185877"
my_page = requests.get(url)


In [20]:
# Use content 
soup = BeautifulSoup(my_page.content, "html.parser")

In [21]:
# Get div of interest
table = soup.find('table', attrs={'class' : 'score'})


In [22]:
trtags = table.find_all('tr',  attrs={'class' : 'match'})

# Helper Function to get information 
def get_info(winner, winnerScore, loser, loserScore): 
    result = {}
    result["winner"] = winner.find('div', attrs={'class': 'name'}).text.strip()
    result["winnerNationality"] = winner.find('div', attrs={'class': 'flag'}).find('img').get("title")
    result["winnerRunouts"] = winner.find('div', attrs={'class': 'runouts'}).text.strip("R")
    if result["winnerRunouts"] == '':
        result["winnerRunouts"] = 0
    result["winnerScore"] = winnerScore

    result["loser"] = loser.find('div', attrs={'class': 'name'}).text.strip()
    result["loserNationality"] = loser.find('div', attrs={'class': 'flag'}).find('img').get("title")
    result["loserRunouts"] = loser.find('div', attrs={'class': 'runouts'}).text.strip("R")
    if result["loserRunouts"] == '':
        result["loserRunouts"] = 0
    result["loserScore"] = loserScore
    return(result)

results_lst = []
for row in trtags: 
    
    # Determine winner and loser
    playerAScore = row.find('td', attrs={'class': 'scoreA'}).find('input').get("value")
    playerBScore = row.find('td', attrs={'class': 'scoreB'}).find('input').get("value")
    if playerAScore > playerBScore: 
        winner = row.find('td', attrs={'class': 'playerA'})
        loser = row.find('td', attrs={'class': 'playerB'})
        result = get_info(winner, playerAScore, loser, playerBScore)
    else: 
        winner = row.find('td', attrs={'class': 'playerB'})
        loser = row.find('td', attrs={'class': 'playerA'})
        result = get_info(winner, playerBScore, loser, playerAScore)
    
    # Get Match Number
    result['matchNum'] = row.find('td', attrs={'class': 'matchno'}).text.strip()
    # Get Match Time
    
    result['matchTime'] = row.find('td', attrs={'class': 'time'}).get('title')
    
    if result['winnerScore'] == "FF": 
        pass
    else: 
        results_lst.append(result)  

    



In [23]:
usopen = pd.DataFrame(results_lst)

In [24]:
usopen["tournament"] = "US Open 9 Ball Championship 2019"

In [25]:
usopen

,winner,winnerNationality,winnerRunouts,winnerScore,loser,loserNationality,loserRunouts,loserScore,matchNum,matchTime,tournament
0,Jayson Shaw,Scotland,2,11,Marcus Westen,Germany,0,0,1,21.Apr 2019 08:43,US Open 9 Ball Championship 2019
1,Jason Williams,United States,0,4,Tom Staveley,United Kingdom,1,11,2,21.Apr 2019 08:55,US Open 9 Ball Championship 2019
2,Hunter Lombardo,United States,0,11,Kosuke Tojo,Japan,2,10,3,21.Apr 2019 08:55,US Open 9 Ball Championship 2019
3,Chad Rhodes,United States,0,4,Radwan Jameel R Sorouji,Saudi Arabia,0,11,4,21.Apr 2019 08:55,US Open 9 Ball Championship 2019
4,Naoyuki Oi,Japan,0,9,Marco Vignola,Italy,0,11,5,21.Apr 2019 08:55,US Open 9 Ball Championship 2019
...,...,...,...,...,...,...,...,...,...,...,...
490,Chang Yu-Lung,Chinese Taipei,0,11,Liu Haitao,China,0,10,499,25.Apr 2019 18:00,US Open 9 Ball Championship 2019
491,Wu Jiaqing,China,0,11,Wang Can,China,0,0,500,25.Apr 2019 20:57,US Open 9 Ball Championship 2019
492,Jeffrey De Luna,Philippines,2,8,Joshua Filler,Germany,0,11,501,26.Apr 2019 09:00,US Open 9 Ball Championship 2019
493,Chang Yu-Lung,Chinese Taipei,0,5,Wu Jiaqing,China,0,11,502,26.Apr 2019 11:30,US Open 9 Ball Championship 2019


In [26]:
players = pd.concat([usopen.winner, usopen.loser] )

In [27]:
players.value_counts()

Joshua Filler            11
Wu Jiaqing               10
Wang Can                 10
Tyler Styer               9
Marco Teutscher           9
                         ..
Thomas Welle              2
Damian Pongpanki          2
Tyler Fleshman            2
Jamie White               1
Fahad Salem N Alharbi     1
Length: 253, dtype: int64

In [30]:
# How many players
nplayers = len(players.value_counts())
nplayers
usopen.winnerScore = usopen.winnerScore.astype(str).astype(int)
usopen.loserScore = usopen.loserScore.astype(str).astype(int)
usopen.matchNum = usopen.matchNum.astype(str).astype(int)

In [31]:
usopen.dtypes

winner               object
winnerNationality    object
winnerRunouts        object
winnerScore           int64
loser                object
loserNationality     object
loserRunouts         object
loserScore            int64
matchNum              int64
matchTime            object
tournament           object
dtype: object

In [32]:
usopen.to_csv("data/usopen2019.csv")

In [87]:
# Plan 
    # Go to https://wpapool.com/ranking/ 
    # look at all the qualifying events
    # Scrape scores from Cuescore 
    # Get priors on Elo system 
    # Use Priors to predict future tournaments 

    

# Turning Stone Classic XXVII

In [33]:
url = "https://cuescore.com/tournament/Turning+Stone+Classic+XXVII+Mid+Season+Championship/1284289"
my_page = requests.get(url)

In [34]:
# Use content 
soup = BeautifulSoup(my_page.content, "html.parser")

In [35]:
# Get div of interest
table = soup.find('table', attrs={'class' : 'score'})


In [36]:
trtags = table.find_all('tr',  attrs={'class' : 'match'})

# Helper Function to get information 
def get_info(winner, winnerScore, loser, loserScore): 
    result = {}
    result["winner"] = winner.find('div', attrs={'class': 'name'}).text.strip()
    result["winnerNationality"] = winner.find('div', attrs={'class': 'flag'}).find('img').get("title")
    result["winnerRunouts"] = winner.find('div', attrs={'class': 'runouts'}).text.strip("R")
    if result["winnerRunouts"] == '':
        result["winnerRunouts"] = 0
    result["winnerScore"] = winnerScore

    result["loser"] = loser.find('div', attrs={'class': 'name'}).text.strip()
    result["loserNationality"] = loser.find('div', attrs={'class': 'flag'}).find('img').get("title")
    result["loserRunouts"] = loser.find('div', attrs={'class': 'runouts'}).text.strip("R")
    if result["loserRunouts"] == '':
        result["loserRunouts"] = 0
    result["loserScore"] = loserScore
    return(result)

results_lst = []
for row in trtags: 
    
    # Determine winner and loser
    playerAScore = row.find('td', attrs={'class': 'scoreA'}).find('input').get("value")
    playerBScore = row.find('td', attrs={'class': 'scoreB'}).find('input').get("value")
    if playerAScore > playerBScore: 
        winner = row.find('td', attrs={'class': 'playerA'})
        loser = row.find('td', attrs={'class': 'playerB'})
        result = get_info(winner, playerAScore, loser, playerBScore)
    else: 
        winner = row.find('td', attrs={'class': 'playerB'})
        loser = row.find('td', attrs={'class': 'playerA'})
        result = get_info(winner, playerBScore, loser, playerAScore)
        
    # Get Match Number
    result['matchNum'] = row.find('td', attrs={'class': 'matchno'}).text.strip()
    # Get Match Time
    
    result['matchTime'] = row.find('td', attrs={'class': 'time'}).get('title')
    
    if result['winnerScore'] == "FF": 
        pass
    else: 
        results_lst.append(result)  


    



In [37]:
turningstone = pd.DataFrame(results_lst)


In [38]:
players = pd.concat([turningstone.winner, turningstone.loser] )
playerlist = players.value_counts().index
playerlist == "Albin Ouschan"

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [39]:
turningstone.to_csv("data/turningstone2017.csv")

In [108]:
turningstone

,winner,winnerNationality,winnerRunouts,winnerScore,loser,loserNationality,loserRunouts,loserScore,matchNum,matchTime
0,Mike Yednak,United States,0,9,Roger Miller,Canada,0,5,1,06.Jan 2017 18:23
1,Johnny Archer,United States,0,9,Jamie Addessi,United States,0,2,2,06.Jan 2017 18:23
2,Jonathan Smith,United States,0,9,Brian Trinci,United States,0,5,3,06.Jan 2017 18:23
3,Oscar Dominguez,United States,0,9,Mike Giurleo,United States,0,3,4,06.Jan 2017 18:23
4,Chris Braiman,United States,0,9,Robert Ferry,United States,0,7,5,06.Jan 2017 18:23
...,...,...,...,...,...,...,...,...,...,...
249,Dennis Hatch,United States,0,9,John Morra,Canada,0,2,250,08.Jan 2017 13:23
250,Jayson Shaw,Scotland,0,9,Rodney Morris,United States,0,3,251,08.Jan 2017 15:01
251,Dennis Hatch,United States,0,9,Amar Kang,India,0,8,252,08.Jan 2017 15:00
252,Rodney Morris,United States,0,9,Dennis Hatch,United States,0,3,253,08.Jan 2017 17:15


# Treviso 

In [122]:
url = "https://www.epbf.com/tournaments/eurotour/draw-results/"
my_page = requests.get(url)
# Use content 
soup = BeautifulSoup(my_page.content, "html.parser")
# Get div of interest
table = soup.find('table', attrs={'class' : 'table'})


In [123]:
table

<table class="table table-sm2 table-dark epbf_table">
<tbody class="round_headline round_all round_1" id="499">
<tr>
<td colspan="11">
<div class="row pt-5">
<div class="col-xl-8 col-md-8 col-sm-8 col-12">
<h3 class="h3">Round 1</h3>
</div>
<div class="col-xl-4 col-md-4 col-sm-4 col-12 text-right">
<div class="custom-control custom-checkbox pr-1 pt-1">
<input class="epbf-control-input walkover_checkbox" id="walkover_checkbox_1" type="checkbox"/>
<label class="epbf-control-label" for="walkover_checkbox_1"><span class="d-none d-sm-block d-md-inherit d-xl-inherit">Show</span> Walkover</label>
</div>
</div>
</div>
</td>
</tr>
</tbody>
<tbody class="round_table table-hover round_all round_1">
<tr>
<tr class="full_layout walkover" id="499">
<td class="info text-center"><span class="d-none d-sm-block d-md-inherit d-xl-inherit">A001</span><span class="d-inherit d-sm-none d-md-none d-xl-none"><span data-container="body" data-content="Date: Jan 1&lt;br&gt;Time: 0:00&lt;br&gt;Table: " data-html="